In [1]:
%run utils.ipynb

In [1]:
class ssf_season:
    """Creates an object representing a single season from a larger dataset.
    Convenient statistics about the season can then be easily accessed as properties.
    
    Parameters:
    year : int
        Year the season began
    standings_data : Pandas dataframe
        Dataframe containing league standings information
    
    """

    def __init__(self, year, standings_data, raw_weekly_info = None):
        _masks = {
            '2015': standings_data['season'] == 2015,
            '2016': standings_data['season'] == 2016,
            '2017': standings_data['season'] == 2017,
            '2018': standings_data['season'] == 2018,
            '2019': standings_data['season'] == 2019,
            '2020': standings_data['season'] == 2020,
            '2021': standings_data['season'] == 2021,
            '2022': standings_data['season'] == 2022,
            '2023': standings_data['season'] == 2023
        }
        self.year = year
        self._raw_season_standings_data = standings_data[_masks[str(year)]]
        _df = self._raw_season_standings_data.copy()
        _df = _df.rename({
                            'name': 'team_name',
                            'team_standings.rank' : 'standings_position',
                            'team_standings.outcome_totals.wins' : 'wins',
                            'team_standings.outcome_totals.losses' : 'losses',
                            'team_standings.outcome_totals.percentage' : 'win_percentage',
                            'team_standings.points_for': 'points_for',
                            'team_standings.points_against' : 'points_against'
                        }, axis = 1)

        # _df['manager_nickname'] = _df['managers.manager'].apply(get_manager_nickname)
        # _df['manager_id'] = _df['managers.manager'].apply(get_manager_id)
        self._season_champion = _df.loc[_df['standings_position']==1, ('team_name','manager_nickname')]
        
        self._standings_data = _df[['standings_position',
                                    'team_name',
                                    'wins',
                                    'losses',
                                    'win_percentage',
                                    'manager_nickname',
                                    'manager_id']]

        self._pf_pa = _df[['standings_position',
                           'team_name',
                           'manager_nickname',
                           'points_for',
                           'points_against']].copy()
        self._pf_pa['point_differential'] = self._pf_pa['points_for'] - self._pf_pa['points_against']
    @property
    def season_champion(self):
        return [self._season_champion['team_name'][0], self._season_champion['manager_nickname'][0]]

    @property
    def season_champion_manager(self):
        return self._season_champion['manager_nickname'][0]

    @property
    def standings_data(self):
        """
        Simplified standings data for the season object
        """
        return self._standings_data.set_index('standings_position')

    @property
    def standings_data_by_win_percentage(self):
        """
        Standings sorted by win percentage descending
        """
        return self._standings_data.sort_values(by='win_percentage', ascending=False).set_index('standings_position')

    @property
    def pf_pa(self):
        """
        Points for and against each team.
        """
        return self._pf_pa.set_index('standings_position')
        